In [1]:
### 1. 특정 경로를 입력받으면 그 경로 안에 있는 xml파일을 다 읽고 
### 아래 함수를 사용해 수정하고 새롭게 저장해서 하위 디렉토리의  폴더에 저장

import os
import xml.etree.ElementTree as ET

##root 를 통해 parsing한 xml을 다시 xml화 시키기 위한 함수
def indent(elem, level=0):
    i = "\n" + level*"  "
    j = "\n" + level*"  "
    if len(elem):
        if not elem.text or not elem.text.strip():
            elem.text = i + "  "
        if not elem.tail or not elem.tail.strip():
            elem.tail = i
        
        for subelem in elem:
            indent(subelem, level+1)
        if not elem.tail or not elem.tail.strip():
            elem.tail = j
    else:
        if level and (not elem.tail or not elem.tail.strip()):
            elem.tail = j
    
    return elem

#path 부분을 Update 하고 새 이름으로 저장하는 함수 [annotation 파일 내의 filename과 이미지파일의 이름이 같아야함.]
def modifiy_annotation(root, darkflow_path, xmlName):
    # filename과 path tag를 find함수로 찾는다.<filename> </filename>  // <path> </path>
    filename = root.find("filename")
    path = root.find("path")    
    folder = root.find("folder")
    width=root.find("width")
    height=root.find("height")
    folder.text = 'dataset'
    data_type = filename.text[-4:]
    image_file_name = xmlName[0:-4]
    # .text는 안의 값을 불러오는 함수이다.
    new_filename =image_file_name+data_type
    #이미지가 저장되어있는 path를 저장 난 darkflow_path+ /data/paper/ 에 이미지가 저장되어있었음
    image_path_dir = darkflow_path + "/data/dataset/"
    new_path = image_path_dir + str(new_filename) 
    path.text = str(new_path) #new_path를 통해 path 의 값을 update한다.
    
    filename.text=new_filename
    
    
    tree =ET.ElementTree(indent(root))
    #난 원본데이터 살리고 싶어서 파일이름을 추가해서 업데이트함. 백업생각할꺼면 이렇게 하는게 나음. 난 label이름 앞에 u 붙임.
    newXmlName = xmlName
    tree.write(newXmlName, xml_declaration=True, encoding='utf-8') # 이걸 써야겠네요 아하 그

    

In [2]:
#이 jupyter notebook은 darkflow 폴더 안에 넣고 실행해야함. 그래야 darkflow_path를 os.getcwd()로 잘 받아옴.
darkflow_path = os.getcwd()

In [3]:
#Annotation을 저장된 위치의 path 나같은 경우엔 darkflow_path +/dataset/dataset_0820/ 이라는 폴더 안에 annotation을 다 저장해두었음
anno_dir=darkflow_path+"/data/XML/"

#tree.write 함수가 현재 폴더에 annotation파일을 저장하기에 현재 쥬피터의 위치를 변경해야함. 새로 저장할 annotation폴더(나 같은 경우 newdataset)을 만들고 저장해야함.
new_anno_dir = darkflow_path+"/data/finalXML/"
os.chdir(new_anno_dir)
os.getcwd()

'C:\\darkflow\\data\\finalXML'

In [4]:
anno_dir

'C:\\darkflow/data/XML/'

In [5]:
os.listdir(anno_dir)

['bean_sprouts_(1).xml',
 'bean_sprouts_(101).xml',
 'bean_sprouts_(102).xml',
 'bean_sprouts_(103).xml',
 'bean_sprouts_(105).xml',
 'bean_sprouts_(109).xml',
 'bean_sprouts_(11).xml',
 'bean_sprouts_(110).xml',
 'bean_sprouts_(112).xml',
 'bean_sprouts_(116).xml',
 'bean_sprouts_(124).xml',
 'bean_sprouts_(125).xml',
 'bean_sprouts_(127).xml',
 'bean_sprouts_(129).xml',
 'bean_sprouts_(130).xml',
 'bean_sprouts_(132).xml',
 'bean_sprouts_(134).xml',
 'bean_sprouts_(136).xml',
 'bean_sprouts_(143).xml',
 'bean_sprouts_(150).xml',
 'bean_sprouts_(151).xml',
 'bean_sprouts_(153).xml',
 'bean_sprouts_(154).xml',
 'bean_sprouts_(155).xml',
 'bean_sprouts_(156).xml',
 'bean_sprouts_(157).xml',
 'bean_sprouts_(160).xml',
 'bean_sprouts_(161).xml',
 'bean_sprouts_(163).xml',
 'bean_sprouts_(164).xml',
 'bean_sprouts_(166).xml',
 'bean_sprouts_(17).xml',
 'bean_sprouts_(174).xml',
 'bean_sprouts_(177).xml',
 'bean_sprouts_(178).xml',
 'bean_sprouts_(182).xml',
 'bean_sprouts_(183).xml',
 'bea

In [6]:
def check_xml_error(root, darkflow_path, xmlName):
    # filename과 path tag를 find함수로 찾는다.<filename> </filename>  // <path> </path>
    filename = root.find("filename")
    path = root.find("path")
    folder = root.find("folder")
    size = root.find('size')
    width = size.find('width')
    height = size.find('height')
    depth = size.find('depth')
    s = [width.text, height.text, depth.text]
    obj = root.findall('object')
    
    
    return filename.text, path.text, folder.text, s, obj

In [7]:
def find_zero(root, darkflow_path, xmlName):
    # filename과 path tag를 find함수로 찾는다.<filename> </filename>  // <path> </path>
    filename = root.find("filename")
    path = root.find("path")
    folder = root.find("folder")
    size = root.find('size')
    width=size.find("width")
    height=size.find("height")
    folder.text = 'dataset'
    
    if width.text == '0' or height.text =='0':
        os.remove(darkflow_path+"/data/finalXML/"+xmlName)

In [8]:
anno_dir

'C:\\darkflow/data/XML/'

In [9]:
from tqdm import tqdm

In [15]:
labels = '''
kimchi
perilla_leaf
beef
carrot
bean_sprouts
steamed_rice
celery
bracken
enoki_mushroom
oyster_mushroom
egg
ramen
potato
tuna_can
'''

In [11]:
cnodir=list()

In [16]:
#os.walk로 anno_dir 안에 있는 파일 이름들을 files에 저장. 그 후 ET.parse와 modifiy_annotation을 통해 path업데이트 후 new_anno_dir에 update된 데이터 저장
for root, dirs, files in os.walk(new_anno_dir):
    for fname in tqdm(files):
#         print(fname)
        tree = ET.parse(new_anno_dir+'/'+fname)
        root2 = tree.getroot()
        xmlName = fname
        modifiy_annotation(root2, darkflow_path, xmlName)
        file_name, path_name, folder_name, size, objs = check_xml_error(root2, darkflow_path, xmlName)
#         find_zero(root2, darkflow_path, xmlName)
#         for name in [file_name, path_name, folder_name]:
#             if ' ' in name or '\t' in name:
#                 print(xmlName)
#         try:
#             with open(path_name,'r') as f:
#                 pass
#         except:
#             cnodir.append(file_name)
#             print(file_name) 
        for obj in objs:
            if not(obj.find('name').text in labels):
                print(xmlName)
        # 여기가 안되는 건가요?
        # 지금 돌려봐도 되죠?넵 아잠시만 하하하하하하
        
#         for obj in objs:
#             if obj.find('name').text in ['mung_bean_sprouts']:
#                 print(xmlName)
#                 obj.find('name').text='bean_sprouts'
#                 tree.write(xmlName, xml_declaration=True, encoding='utf-8') # Good
                
len(cnodir)        

 37%|████████████████████████████▊                                                | 2075/5546 [00:05<00:08, 390.90it/s]

celery_3_0579.xml
celery_3_0579.xml
celery_3_0579.xml
celery_3_0579.xml


 51%|███████████████████████████████████████▎                                     | 2829/5546 [00:07<00:06, 400.83it/s]

kimchi_116.xml
kimchi_116.xml
kimchi_127.xml
kimchi_127.xml
kimchi_127.xml
kimchi_132.xml
kimchi_132.xml


 52%|███████████████████████████████████████▊                                     | 2870/5546 [00:07<00:07, 380.32it/s]

...

mung_bean_sprouts_136.xml


 66%|██████████████████████████████████████████████████▌                          | 3646/5546 [00:09<00:04, 394.27it/s]

mung_bean_sprouts_39.xml
mung_bean_sprouts_39.xml
mung_bean_sprouts_39.xml
mung_bean_sprouts_64.xml
mung_bean_sprouts_64.xml


100%|█████████████████████████████████████████████████████████████████████████████| 5546/5546 [00:14<00:00, 391.23it/s]


0

In [13]:
cnodir

[]